# Subsetter Panel App

Goal: interactively choose a small bounding box to download as netCDF file for local QGIS work

NOTE: currently lacking download capabilities

In [ ]:
import intake
import satsearch
import xarray as xr
import rioxarray 
import os
import dask
import pandas as pd

# visualization
import holoviews as hv
import hvplot.xarray
import panel as pn
import param

In [ ]:
# Initialization steps not requiring auth
# NOTE: streaming with GDAL from NSIDC SERVER REQUIRES you have a ~/.netrc file 
# behind the scenes we're using GDAL to make requests, and we set some Env vars for performance
#GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           GDAL_HTTP_COOKIEFILE='.urs_cookies',
           GDAL_HTTP_COOKIEJAR='.urs_cookies',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
           GDAL_SWATH_SIZE='200000000',
           VSI_CURL_CACHE_SIZE='200000000')
os.environ.update(env)

# Hard code greenland gamma0 mosaics for now
def get_STAC_items():
    url='https://cmr.earthdata.nasa.gov/stac/NSIDC_ECS'
    collection = 'C1908075185-NSIDC_ECS'
    #dates = '2014-01-01/2020-12-31' 
    #bbox = [lonmin, latmin, lonmax, latmax] 
    results = satsearch.Search.search(url=url,
                        collections=[collection], 
                        #datetime=dates,
                        #bbox=bbox,    
                        sortby=['+properties.datetime'])

    items = results.items()
    print(f'Found {len(items)} STAC Items')

    return intake.open_stac_item_collection(items)

# No great way to filter by asset type currently (gamma0, sigma0, thumbnail), so need a function
def filter_assets(catalog, key='1', pattern='gamma0'):
    all_assets = [item.assets.get(key) for item in catalog._stac_obj]
    all_hrefs = [asset.get('href') for asset in all_assets if asset] #ignore None values if asset key missing
    filtered = [i for i in all_hrefs if pattern in i]
    print(f'{len(filtered)} Assets match pattern {pattern}')
    
    return filtered

    
catalog = get_STAC_items()
assets = filter_assets(catalog)

In [ ]:
class Stage1(param.Parameterized):
    
    # widget-linked variables
    username = param.String()
    password = param.String()
    #ready = param.Boolean(default=False, precedence=-1)
    action = param.Action(lambda x: x.param.trigger('action'), label='Enter Credentials')

    @param.depends('action', watch=True)
    def _write_netrc(self):
        #self.ready = True
        
        #print(self.username)
        # NOTE: event is from linked button
        # write.netrc file if it doesn't exist (on mybinder.org)
        netrcPath = '/home/jovyan/.netrc'
        if not os.path.exists(netrcPath):
            with open(netrcPath, 'w') as f:
                f.write(f'urs.earthdata.nasa.gov {self.username} {self.password}')
        os.chmod(netrcPath, 0o600)    
    
    @param.output(('state', param.Boolean))
    def output(self):
        return self.ready
    
    def view(self):
         # view depending on widget values, doesn't really matter here...
        text = pn.pane.Markdown('''
        ## MEaSUREs Greenland Image Mosaics from Sentinel-1A and -1B, Version 3
        *from Copernicus Sentinel-1A and -1B imaging satellites starting in January 2015*

        <a href="https://nsidc.org/data/nsidc-0723" target="_blank">Dataset technical reference (nsidc-0723)</a>

        <a href="http://epsg.io/3413" target="_blank">Map projection reference (EPSG:3413)</a>

        **Instructions: Click 'Enter Credentials button', then click 'Next' to bring up App**
        **Be patient, the visualization takes a minute to load**
        ''', width=800)   
        return text
    
    def panel(self):
        widgets = pn.panel(self.param,  widgets={'password': pn.widgets.PasswordInput})
        return pn.Row(widgets, self.view)

In [ ]:
#stage1 = Stage1(name='NASA Earthdata credentials:')
#stage1.panel()

In [ ]:
# Only need to construct dataset once
# NOTE: ensure init doesn't happen until required

@dask.delayed
def lazy_open(href, masked=True):
    filename = href.split('/')[-1] 
    date = href.split('/')[-2] 
    da = rioxarray.open_rasterio(href, chunks=(1, "auto", -1), masked=masked).rename(band='time') 
    da['time'] = [pd.to_datetime(date)]
    da['filename'] = filename
    return da

class Stage2(param.Parameterized):
    
    auth_ready = param.Boolean(default=True, precedence=-1)    
    box = hv.streams.BoundsXY(bounds=(-243538, -2295690, -149311, -2254858)) #jakobs
    #print(type(DA))

    def __init__(self, **params):
        super().__init__(**params)
        # Seems single-machine scheduler uses threads by default (ThreadPool), you can use processes instead (ProcessPool)
        with dask.config.set(scheduler='processes'): 
            dataArrays = dask.compute(*[lazy_open(href, masked=False) for href in assets])

        # NOTE: this is fast with dask arrays, can run out of memory with numpy arrays
        # NOTE: this needs to be within init to only run when needed
        self.DA = xr.concat(dataArrays, dim='time', join='override', combine_attrs='drop')
    
    
    def plot_map(self):
        # xmin=-625975, xmax=849975, ymin=-3355975 ymax=-695025  .redim.range(x=(-625975, 849975), y=(-695025, -335597)
        #extent = (-625975, -3355975, 849975, -695025) #(minx, miny, maxx, maxy)
        
        da = rioxarray.open_rasterio(assets[-1], chunks=(1, "auto", -1), overview_level=2, masked=False).squeeze('band') 
        img = da.hvplot.image(rasterize=True, cmap='gray', aspect='equal', frame_width=400)
        self.box.source = img
        bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[self.box]).opts(color='red')
        mapview = pn.Column(img * bounds) 
        #print(type(mapview))
        return mapview
    
    
    @pn.depends(box.param.bounds)
    def plot_video(self, data):
        #DA = load_data(assets)
        keys = ['minx','miny','maxx','maxy']
        bbox_dict = dict(zip(keys,data))
        subset = self.DA.rio.clip_box(**bbox_dict)
        video = subset.hvplot.image(x='x',y='y', 
                                rasterize=True,
                                cmap='gray', clim=(-25,5),
                                aspect='equal', frame_width=800,
                                widget_type='scrubber', widget_location='bottom') 

        widget = video[1][1][0] 
        widget.interval = 2000   #2 sec between frames 500 ms default
        #print(type(video))
        return video
    
    def view(self):
        return pn.Row(self.plot_map, self.plot_video)
    
    # no parameteres in this case...
    def panel(self):    
        return self.view()

In [ ]:
#stage2 = Stage2()
#stage2.panel()

In [ ]:
# # NOTE: for some reason, extent is off when putting image through pipeline...
# add it to the pipeline
pipeline = pn.pipeline.Pipeline()
pipeline.add_stage('Authenticate', Stage1(name='NASA EarthData Credentials'))
pipeline.add_stage('Visualize', Stage2)
pipeline.layout.servable()